In [8]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau
import visualkeras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.layers import multiply  # Import the multiply function


In [2]:
top5_acc = TopKCategoricalAccuracy(k=5, name='top5_acc')
top10_acc = TopKCategoricalAccuracy(k=10, name='top10_acc')

In [3]:
root_path = r"C:\Users\Jarushen\Desktop\Masters Thesis\Images\Fynbos"  # Your dataset path

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Adjust for ResNet50
    rotation_range=10,  # Reduced rotation
    width_shift_range=0.1,  # Reduced shift range
    height_shift_range=0.1,  # Reduced shift range
    shear_range=0.1,  # Reduced shear range
    zoom_range=0.1,  # Reduced zoom range
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


# Load images from directory
train_generator = datagen.flow_from_directory(
    root_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    root_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 706 images belonging to 23 classes.
Found 163 images belonging to 23 classes.


In [14]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, Multiply, Activation, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the Squeeze-and-Excitation block
def squeeze_excite_block(input_tensor, ratio=16):
    # Get the number of channels (filters) in the input tensor
    channels = input_tensor.shape[-1]
    
    # Squeeze: Global average pooling
    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape((1, 1, channels))(se)
    
    # Excitation: Two fully connected layers
    se = Dense(channels // ratio, activation='relu', use_bias=False)(se)
    se = Dense(channels, activation='sigmoid', use_bias=False)(se)
    
    # Scale the input tensor by the SE block output
    x = Multiply()([input_tensor, se])
    return x

# Load the base model, pre-trained on ImageNet
input_tensor = Input(shape=(150, 150, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Unfreeze the last 30 layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add the SE block to the base model's output
x = base_model.output
x = squeeze_excite_block(x)

# Continue with the rest of the network
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

# You need to ensure train_generator is already defined before this line
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model (assuming you have already defined the compile parameters)


# Compile the model with a reduced initial learning rate
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_k_categorical_accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    epochs=10,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)




Epoch 1/10
23/23 [==============================] - 1809s 79s/step - loss: 2.8414 - acc: 0.1657 - top_k_categorical_accuracy: 0.4575 - val_loss: 2.4560 - val_acc: 0.2331 - val_top_k_categorical_accuracy: 0.6196
Epoch 2/10
23/23 [==============================] - 811s 35s/step - loss: 1.8073 - acc: 0.5524 - top_k_categorical_accuracy: 0.8456 - val_loss: 1.8284 - val_acc: 0.4049 - val_top_k_categorical_accuracy: 0.7178
Epoch 3/10
23/23 [==============================] - 783s 34s/step - loss: 1.0834 - acc: 0.7564 - top_k_categorical_accuracy: 0.9603 - val_loss: 1.5121 - val_acc: 0.5215 - val_top_k_categorical_accuracy: 0.8528
Epoch 4/10
23/23 [==============================] - 792s 34s/step - loss: 0.7053 - acc: 0.8555 - top_k_categorical_accuracy: 0.9858 - val_loss: 1.3261 - val_acc: 0.5890 - val_top_k_categorical_accuracy: 0.8773
Epoch 5/10
23/23 [==============================] - 783s 34s/step - loss: 0.4116 - acc: 0.9221 - top_k_categorical_accuracy: 0.9915 - val_loss: 1.2860 - val_ac

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/25
23/23 [==============================] - 845s 37s/step - loss: 2.9306 - acc: 0.2025 - top5_acc: 0.4958 - top10_acc: 0.6969 - val_loss: 322.0605 - val_acc: 0.0613 - val_top5_acc: 0.9571 - val_top10_acc: 0.9877
Epoch 2/25
23/23 [==============================] - 933s 41s/step - loss: 2.2788 - acc: 0.3499 - top5_acc: 0.7280 - top10_acc: 0.9023 - val_loss: 3448.7728 - val_acc: 0.0491 - val_top5_acc: 1.0000 - val_top10_acc: 1.0000
Epoch 3/25
18/23 [======================>.......] - ETA: 3:06 - loss: 1.7748 - acc: 0.4844 - top5_acc: 0.8542 - top10_acc: 0.9549

KeyboardInterrupt: 